In [1]:
import pandas as pd

In [2]:
# NOTE: For consistency sake, I am making sure that whatever the user wants (city with high pop, low age, high income)
# that those ranked columns output large numbers. So wanting a large city will be fine since Philly will rank out as a
# high number, but wanting a small city, I invert the rank so the smallest city in PA, Baileyville, is now the largest #

# Method to sort and rank cities and counties based on what the user selects in our GUI
# data1-3 are the columns the user cares about, d1-3Flag are whether the user cares about smaller or larger values
# and cflag is whether the user is looking at cities or counties
def main(data1, d1Flag, data2, d2Flag, data3, d3Flag, cflag):
    df = pd.DataFrame(pd.read_csv('Master_Data_File.csv', sep=',', encoding='utf-8'))
    if cflag == 'City':
        # Create sorted grouping of columns based on users interests
        group1 = df.sort_values(by=data1, ascending=d1Flag)
        group2 = df.sort_values(by=data2, ascending=d2Flag)
        group3 = df.sort_values(by=data3, ascending=d3Flag)

        # Compute the rank of the values in that column, I negate the flag since I want everything weighted the same
        rank1 = df[data1].rank(ascending=not d1Flag)
        rank2 = df[data2].rank(ascending=not d2Flag)
        rank3 = df[data3].rank(ascending=not d3Flag)

        agggroup = pd.DataFrame(data=[df['City'], rank1 + rank2 + rank3], index=['City', 'Rank']).T.sort_values(by='Rank', ascending=False)
    else:  # County
        # Some of our columns need to be summed and not averaged, making a list of them here
        sumList = ['Population', 'White Alone', 'Hispanic', 'Two or more races', 'Native Hawaiian', 'Black alone',
                   'Asian alone', 'American Indian alone', 'Other race alone']
        # Some of data is being assessed as Object type, even though it is clearly an int, put in this failsafe
        df[data1] = df[data1].astype('float64')
        df[data2] = df[data2].astype('float64')
        df[data3] = df[data3].astype('float64')
        group = df.groupby('County')

        # Set of if statements to assess if the input data should be summed or averaged
        if data1 in sumList:
            group1 = group[data1].sum().sort_values(ascending=d1Flag)
        else:
            group1 = group[data1].mean().sort_values(ascending=d1Flag)
        if data2 in sumList:
            group2 = group[data2].sum().sort_values(ascending=d2Flag)
        else:
            group2 = group[data2].mean().sort_values(ascending=d2Flag)
        if data3 in sumList:
            group3 = group[data3].sum().sort_values(ascending=d3Flag)
        else:
            group3 = group[data3].mean().sort_values(ascending=d3Flag)

        # Compute the rank of the values in that column, I negate the flag since I want everything weighted the same
        rank1 = group1.rank(ascending=not d1Flag)
        rank2 = group2.rank(ascending=not d2Flag)
        rank3 = group3.rank(ascending=not d3Flag)

        agggroup = pd.DataFrame(data=[rank1 + rank2 + rank3], index=['Rank']).T.sort_values(by='Rank', ascending=False)

    return group1, group2, group3, agggroup

In [ ]:
if __name__ == '__main__':
    main('Median House Value', False, 'Average Income', False, 'Population', False, 'County')